<a href="https://colab.research.google.com/github/cotozelo/Precisa_Operacional/blob/main/AHP_e_Evolu%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Um estudo do método AHP e suas variações

O método AHP (Analytic Hierarchy Process) faz parte uma área de pesquisa chamada Pesquisa Operacional. 

Nesse estudo vamos apresentar o método na sua integrar e apresentar uma proposta chamada AHP-Gaussino, além de propor uma simplificação no método AHP-Gaussiano.

Foi usado para esse estudo o vídeo: https://www.youtube.com/watch?v=ydeNa_-l6vs

## A pergunta

Os métodos AHP começam com uma pergunta a ser respondida. Essa pergunta deve ser geralmente é alinha a uma escolhar, exemplo: qual celular comprar, qual carro é melhor, que setor é menos produtivo...

Nesse estudo iremos responder a seguinte pergunta:

**<h3> Qual o carro com melhor custo benefício?</h3>**

In [58]:
#@title Tabela dos carros
import pandas as pd

df_carros = pd.DataFrame({
'Nome':          ['Fiesta', 'Uno', 'Gol', 'March', 'Sandero'  ],
'Preco [R$]':    [31000   , 30000, 35000,  32000,   31000],
'Potencia [cv]': [107     ,    88,    75,    111,   107],
'Motor [L]':     [1.6     ,   1.4,   1.0,    1.6,   1.6],
'Torque [kgfm]': [15.3    ,  12.3,  10.6,   15.1,  15.3]})

df_carros

,Nome,Preco [R$],Potencia [cv],Motor [L],Torque [kgfm]
0,Fiesta,31000,107,1.6,15.3
1,Uno,30000,88,1.4,12.3
2,Gol,35000,75,1.0,10.6
3,March,32000,111,1.6,15.1
4,Sandero,31000,107,1.6,15.3


In [59]:
#@title Valores do critérios dos carro normalizados 

df_car_norm = df_carros.copy()

preco = 1/df_car_norm['Preco [R$]']
preco.sum()

df_car_norm['Preco [R$]'] = preco/preco.sum()
df_car_norm['Potencia [cv]'] = df_car_norm['Potencia [cv]']/df_car_norm['Potencia [cv]'].sum()
df_car_norm['Motor [L]'] = df_car_norm['Motor [L]']/df_car_norm['Motor [L]'].sum()
df_car_norm['Torque [kgfm]'] = df_car_norm['Torque [kgfm]']/df_car_norm['Torque [kgfm]'].sum()
df_car_norm

,Nome,Preco [R$],Potencia [cv],Motor [L],Torque [kgfm]
0,Fiesta,0.204591,0.219262,0.222222,0.223032
1,Uno,0.211411,0.180328,0.194444,0.179300
2,Gol,0.181209,0.153689,0.138889,0.154519
3,March,0.198198,0.227459,0.222222,0.220117
4,Sandero,0.204591,0.219262,0.222222,0.223032


## AHP

O método AHP contém uma escala fundamental de gráu de importância entre os critérios:


Grau de importância | Descrição da Relação
-----|--------------------------
1    | Igualdade
2    | Entre adjacentes
3    | Importância moderada
4    | Entre adjacentes
5    | Mais importante
6    | Entre adjacentes
7    | Muito mais importante
8    | Entre adjacentes
9    | Extremamente importante

### Construindo a matriz de importância

A cada par de critério (Preço, Potência, Moto, Torque) atribuimos um grau de importância. 

A diagonal da matriz é 1.

Lembrando de manter a coerência: Se A > B e B > C então A > C

Essa etapa é preciso da intervenção de um especialista (ou interessado), quanto maior a quantidade de critérios, mais trabalhosa é a etapa.

A matriz não pode ser preenchida na sua totalidade. Pois ao atribuir um valor para um elemento 
$a_{ij} = x$ 
o elemento 
$a_{ji} = 1/x$ 
atribuído automaticamente, esse elemento chama elemento de reciprocidade.

No caso desse exemplo vou pontar o que seria melhor para mim na escolha de um carro.

In [60]:
#@title Atribuindo grau de importância:
df_importancia = pd.DataFrame({
    'Nome'     : ['Torque', 'Potência', 'Motor', 'Preço'],
    'Torque'   : [1       ,         '',      '',      ''],
    'Potência' : [3       ,          1,      '',      ''],
    'Motor'    : [5       ,          3,       1,      ''],   
    'Preço'    : [7       ,          7,       3,       1]
})
df_importancia 

,Nome,Torque,Potência,Motor,Preço
0,Torque,1,3,5,7
1,Potência,,1,3,7
2,Motor,,,1,3
3,Preço,,,,1


In [61]:
#@title Calculando elementos reciprocos:
df_importancia = pd.DataFrame({
    'Nome'     : ['Torque', 'Potência', 'Motor', 'Preço'],
    'Torque'   : [1       ,        1/3,     1/5,     1/7],
    'Potência' : [3       ,          1,     1/3,     1/7],
    'Motor'    : [5       ,          3,       1,     1/3],   
    'Preço'    : [7       ,          7,       3,       1]
})

df_importancia['Preço'] = df_importancia['Preço'].astype(float)
df_importancia['Potência'] = df_importancia['Potência'].astype(float)
df_importancia['Motor'] = df_importancia['Motor'].astype(float)
df_importancia['Torque'] = df_importancia['Torque'].astype(float)
df_importancia

,Nome,Torque,Potência,Motor,Preço
0,Torque,1.000000,3.000000,5.000000,7.0
1,Potência,0.333333,1.000000,3.000000,7.0
2,Motor,0.200000,0.333333,1.000000,3.0
3,Preço,0.142857,0.142857,0.333333,1.0


In [62]:
#@title Matriz de importância normalizada: <br> <h5>Normalização na linha</h5>
df_import_normal = df_importancia.copy()

for col in ['Preço', 'Potência', 'Motor', 'Torque']:
    df_import_normal[col] = df_import_normal[col]/df_import_normal[col].sum()

df_import_normal

,Nome,Torque,Potência,Motor,Preço
0,Torque,0.596591,0.670213,0.535714,0.388889
1,Potência,0.198864,0.223404,0.321429,0.388889
2,Motor,0.119318,0.074468,0.107143,0.166667
3,Preço,0.085227,0.031915,0.035714,0.055556


In [63]:
#@title Criando vetor de prioridade <br> <h5>Soma da linha da matriz de importância normalizada</h5>
df_vet_prioridade = df_import_normal.copy()
df_vet_prioridade = df_vet_prioridade.drop(columns=["Nome"]).T[:].sum()/df_vet_prioridade.drop(columns=["Nome"]).T[:].shape[0]
df_vet_prioridade.index = list(df_import_normal.columns)[1:]
df_vet_prioridade = df_vet_prioridade.to_frame().T
df_vet_prioridade

,Torque,Potência,Motor,Preço
0,0.547852,0.283146,0.116899,0.052103


### Calculando a consistência na imputação dos graus de importância

Onde multiplicamos a matrix de importância (sem normalizar) pelo vetor de prioridade.

In [64]:
#@title Calculando C.R.
df_teste = df_importancia.copy()
imp_prio = df_teste.drop(columns='Nome').to_numpy() * df_vet_prioridade.T[0].to_numpy()

lambda_max = 0
for ii in range(df_importancia.shape[0]):
    lambda_max += imp_prio[ii,:].sum()/df_vet_prioridade.T[0].to_numpy()[ii]
    
lambda_max = lambda_max/df_importancia.shape[0]
print()
ri = 0.9
ci = (lambda_max - df_importancia.shape[0])/(df_importancia.shape[0]-1)
cr = (ci / ri)*100
print(f'Lambda Máximo: {lambda_max}\n')
print(f'R.I.: {ri}\n')
print(f'C.I.: {ci}\n')
print(f'C.R.: {cr}% {"Grau imputado ERRADO, valor maior que 10%" if cr > 10 else "Grau imputado Correto"}')
print('\nEssa é a etapa mais complicada do método, encontrar um C.R. que seja menor que 10%.\nQuando o valor excede a 10% devemos revisitar os Graus de importâncias imputados.')


Lambda Máximo: 4.140718216719034

R.I.: 0.9

C.I.: 0.04690607223967813

C.R.: 5.211785804408682% Grau imputado Correto

Essa é a etapa mais complicada do método, encontrar um C.R. que seja menor que 10%.
Quando o valor excede a 10% devemos revisitar os Graus de importâncias imputados.


### Normalizando a tabela com os modelos dos carros

Para a normalização seguimos as seguintes equações:

* influenciam positivamente: $v_{ij} = \frac{a_{ij}}{\sum_{j=0}^{n}a_{ij}}$ 

* influenciam negativamente: $v_{ij} = \frac{\frac{1}{a_{ij}}}{\sum_{j=0}^{n}\frac{1}{a_{ij}}}$ 

---
* influenicam Negativamente:
 * Preço

* influenicam Positivamente:
 * Potência
 * Torque
 * Motor

In [65]:
#@title Valores normalizados vezes o vetor de prioridade

df_car_import = df_car_norm.copy()
df_car_import['Preco [R$]'] = df_car_norm['Preco [R$]'] * df_vet_prioridade['Preço'].iloc[0]
df_car_import['Potencia [cv]'] = df_car_norm['Potencia [cv]'] * df_vet_prioridade['Potência'].iloc[0]
df_car_import['Motor [L]'] = df_car_norm['Motor [L]'] * df_vet_prioridade['Motor'].iloc[0]
df_car_import['Torque [kgfm]'] = df_car_norm['Torque [kgfm]'] * df_vet_prioridade['Torque'].iloc[0]
df_car_import

,Nome,Preco [R$],Potencia [cv],Motor [L],Torque [kgfm]
0,Fiesta,0.010660,0.062083,0.025978,0.122189
1,Uno,0.011015,0.051059,0.022730,0.098230
2,Gol,0.009442,0.043516,0.016236,0.084653
3,March,0.010327,0.064404,0.025978,0.120591
4,Sandero,0.010660,0.062083,0.025978,0.122189


In [66]:
#@title Somando a linha e encontrando o score e o melhor custo benefício
df_car_custo_beneficio = pd.DataFrame({
    'Modelo': ['Fiesta', 'Uno', 'Gol', 'March', 'Sandero'],
    'Score':  [df_car_import[df_car_import['Nome'] == 'Fiesta'].T[0].iloc[1:].sum(),
               df_car_import[df_car_import['Nome'] == 'Uno'].T[1].iloc[1:].sum(),
               df_car_import[df_car_import['Nome'] == 'Gol'].T[2].iloc[1:].sum(),
               df_car_import[df_car_import['Nome'] == 'March'].T[3].iloc[1:].sum(),
               df_car_import[df_car_import['Nome'] == 'Sandero'].T[4].iloc[1:].sum()]})
df_car_custo_beneficio.sort_values('Score', inplace=True, ascending=False)
df_car_custo_beneficio

,Modelo,Score
3,March,0.221300
0,Fiesta,0.220909
4,Sandero,0.220909
1,Uno,0.183035
2,Gol,0.153847


Melhor custo benefício é o March, seguindo por Fiesta, Sandero, Uno e Gol, respectivamente.

## AHP Gaussiano & AHP Sigma

A principal diferença entre os métodos AHP tradicional, para os dois apresentados a seguir. É a não necessidade de um especialista para imputar o grau de relevância entre os critérios, sem essa necessidade o método pode ser automazado, e facilita ainda mais sua usabilidade.

Calcular o desvio padrão de cada critério:

In [67]:
#@title Desvio Padrão (sigma)
sigma = df_car_norm.describe().loc['std', :].to_dict()
sigma

{'Preco [R$]': 0.011496775635213294,
 'Potencia [cv]': 0.031732534973851395,
 'Motor [L]': 0.03621779114001473,
 'Torque [kgfm]': 0.031474730406029}

In [68]:
#@title Média
media = df_car_norm.describe().loc['mean', :].to_dict()
media

{'Preco [R$]': 0.2,
 'Potencia [cv]': 0.2,
 'Motor [L]': 0.20000000000000004,
 'Torque [kgfm]': 0.19999999999999998}

### Diferença entre AHP Gaussiano & AHP Sigma

O AHP Gaussiano usa o fator gaussiano: $f_{i}=\frac{\sigma_{i}}{média_{i}}$

In [69]:
#@title Fator Gaussiano
fator_gaussiano = {}
soma_fator_gaussiano = 0
for kk in sigma:
    fator_gaussiano[kk] = sigma[kk]/media[kk]
    soma_fator_gaussiano += fator_gaussiano[kk]
fator_gaussiano    

{'Preco [R$]': 0.05748387817606647,
 'Potencia [cv]': 0.15866267486925698,
 'Motor [L]': 0.1810889557000736,
 'Torque [kgfm]': 0.15737365203014503}

Calculando a normalização de proporcionalidade no Fator Gaussiano: $fn_{i}=\frac{f_{i}}{\sum{f_{i}}}$


In [70]:
#@title Fator Gaussiano Normalizado
fator_gaussiano_normal = {}
for kk in sigma:
    fator_gaussiano_normal[kk] = fator_gaussiano[kk]/soma_fator_gaussiano
fator_gaussiano_normal  

{'Preco [R$]': 0.10364754540960598,
 'Potencia [cv]': 0.28608015534289005,
 'Motor [L]': 0.3265163443150604,
 'Torque [kgfm]': 0.2837559549324435}

Para o Método AHP Sigma calculamos a normalização do desvio padrão: $\sigma n_{i}=\frac{\sigma_{i}}{\sum{\sigma_{i}}}$

In [71]:
#@title Desvio Padrão Normalizado ($\sigma n$)
sigma_normal = {}
sigma_soma = sum(list(sigma.values()))
for kk in sigma:
    sigma_normal[kk] = sigma[kk]/sigma_soma
sigma_normal  

{'Preco [R$]': 0.103647545409606,
 'Potencia [cv]': 0.28608015534289005,
 'Motor [L]': 0.3265163443150605,
 'Torque [kgfm]': 0.28375595493244343}

Note que os valores do Fator Gaussiano Normalizado e do Sigma Normalizado, são exatamente iguais. 

Logo não há necessidade de calcular o Fator Gaussiano, basta usar o Desvio Padrão.

Com a proposta de usar o desvio padrão (somente) veio a evolução para o método AHP Sigma

In [72]:
#@title Valores normalizados vezes o sigma normal

df_car_import_sigma = df_car_norm.copy()
df_car_import_sigma['Preco [R$]'] = df_car_norm['Preco [R$]'] * sigma_normal['Preco [R$]']
df_car_import_sigma['Potencia [cv]'] = df_car_norm['Potencia [cv]'] * sigma_normal['Potencia [cv]']
df_car_import_sigma['Motor [L]'] = df_car_norm['Motor [L]'] * sigma_normal['Motor [L]']
df_car_import_sigma['Torque [kgfm]'] = df_car_norm['Torque [kgfm]'] * sigma_normal['Torque [kgfm]']
df_car_import_sigma

,Nome,Preco [R$],Potencia [cv],Motor [L],Torque [kgfm]
0,Fiesta,0.021205,0.062727,0.072559,0.063287
1,Uno,0.021912,0.051588,0.063489,0.050878
2,Gol,0.018782,0.043967,0.045349,0.043846
3,March,0.020543,0.065072,0.072559,0.062459
4,Sandero,0.021205,0.062727,0.072559,0.063287


In [73]:
#@title Somando a linha e encontrando o score e o melhor custo benefício
df_car_custo_beneficio_sigma = pd.DataFrame({
    'Modelo': ['Fiesta', 'Uno', 'Gol', 'March', 'Sandero'],
    'Score':  [df_car_import_sigma[df_car_import_sigma['Nome'] == 'Fiesta'].T[0].iloc[1:].sum(),
               df_car_import_sigma[df_car_import_sigma['Nome'] == 'Uno'].T[1].iloc[1:].sum(),
               df_car_import_sigma[df_car_import_sigma['Nome'] == 'Gol'].T[2].iloc[1:].sum(),
               df_car_import_sigma[df_car_import_sigma['Nome'] == 'March'].T[3].iloc[1:].sum(),
               df_car_import_sigma[df_car_import_sigma['Nome'] == 'Sandero'].T[4].iloc[1:].sum()]})
df_car_custo_beneficio_sigma.sort_values('Score', inplace=True, ascending=False)
df_car_custo_beneficio_sigma

,Modelo,Score
3,March,0.220633
0,Fiesta,0.219778
4,Sandero,0.219778
1,Uno,0.187867
2,Gol,0.151944


## Coparando resultado do AHP com AHP Sigma

In [74]:
#@title AHP
df_car_custo_beneficio

,Modelo,Score
3,March,0.221300
0,Fiesta,0.220909
4,Sandero,0.220909
1,Uno,0.183035
2,Gol,0.153847


In [75]:
#@title AHP Sigma
df_car_custo_beneficio_sigma

,Modelo,Score
3,March,0.220633
0,Fiesta,0.219778
4,Sandero,0.219778
1,Uno,0.187867
2,Gol,0.151944


Vermos que há diferença entre os resultados são entre 2,5% e 0,3%. Mas a ordenação se preserva. 

E o mais importante NÃO foi preciso usar um especialista para atribuir elação de importancia entre os critérios. 